# Synthetic Aperature Radar (SAR) Satellite Location Demo

## Summary

This notebook visualizes select SAR satellite orbit paths over Myanmar and Thailand from when the code is run until 48h later, to aid in humanitarian response to the region.

**It does not guarantee a SAR image has been taken. Nor does it incldue all SAR satellites. It can be used to determine where images are likely to be captured, and by whom.**

Please click on `Run` in the menu bar, followed by `Run All Cells`. You'll see an `*` next to the cells that are currently or waiting to be run, and a number once the cell has run. If you see a blank bracket `[ ]`, please re-run all cells again. 

If the last cell doesn't show a map, please re-run that cell by selecting it and then pressing `shift + enter`

Cells have been hidden to focus on the output. Please click on any section or cell to expand it.

## Background

Timeliness is an important aspect for countless uses of Earth Observation (EO) data. Humanitarians and emergency response organizations, for example, benefit from imagery immediately before or after an emergency.

Knowing when an image was captured of an area of interested, or when it will be captured next, can help humanitarians emergency response organizations better understand where aid is most needed.

This notebook demonstrates how the location of synthetic aperature radar (SAR) EO satellites can be computed in an area of interest.

## Load Libraries and Data

We'll use the [sgp4](https://pypi.org/project/sgp4/) and [skyfield](https://rhodesmill.org/skyfield/) libraries, which help us calculate the position of satellites using complex orbital physics. Shapely will help us with some of the geometry creation and calculations, and Lonboard will be used to visualize the results.

In [53]:
import micropip
await micropip.install("https://ds-wheels.s3.amazonaws.com/sgp4-2.23-cp312-cp312-pyodide_2024_0_wasm32.whl")

%pip install pyodide-unix-timezones

In [54]:
deps = [
    "https://ds-wheels.s3.amazonaws.com/pyarrow-17.0.0-cp312-cp312-pyodide_2024_0_wasm32.whl",
    "https://ds-wheels.s3.amazonaws.com/arro3_core-0.3.0-cp312-cp312-emscripten_3_1_58_wasm32.whl",
    "https://ds-wheels.s3.amazonaws.com/arro3_compute-0.3.0-cp312-cp312-emscripten_3_1_58_wasm32.whl",
    "https://ds-wheels.s3.amazonaws.com/arro3_io-0.3.0-cp312-cp312-emscripten_3_1_58_wasm32.whl",
    "https://ds-wheels.s3.amazonaws.com/geoarrow_rust_core-0.3.0b1-cp38-abi3-emscripten_3_1_58_wasm32.whl",
    "palettable",
    "matplotlib",
    "lonboard==0.10.0b2"
]
await micropip.install(deps)

In [55]:
%pip install skyfield

In [56]:
from skyfield.api import load, wgs84, Timescale

from datetime import datetime, timedelta, timezone

from lonboard import viz, Map, ScatterplotLayer, basemap, PathLayer
from lonboard.colormap import apply_continuous_cmap
from palettable.colorbrewer.sequential import Oranges_9

import geopandas as gpd
import pandas as pd
from shapely import Point, LineString, Polygon
from shapely.geometry import LineString

import requests

## Set up list of satellites, area of interest (AOI) and timeframe

We'll load information about specific satellites from [Celestrak](https://celestrak.org). Due to the complexities of orbital physics, data about satellites needs to be updated frequently is only accurate for about 1 week before and after "epoch".

This demo fetches the orbital predictions for some (but not all) SAR satellites.

In [57]:
urls = [
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2014-016", # SENTINEL-1A
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2024-235", # SENTINEL-2C
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2007-061", # RADARSAT-2
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2012-017", # RISAT-1
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2007-023A", # COSMO-SKYMED 1
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2007-059A", # COSMO-SKYMED 2
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2008-054A", # COSMO-SKYMED 3
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2010-060A" # COSMO-SKYMED 4
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2018-099AU",  # ICEYE-X2,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2019-038C",  # ICEYE-X5,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2019-038D",  # ICEYE-X4,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2020-068L",  # ICEYE-X7,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2020-068M",  # ICEYE-X6,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2021-006CY",  # XR-1 (ICEYE-X10),
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2021-006DB",  # ICEYE-X8,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2021-059AP",  # ICEYE-X13,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2021-059AQ",  # ICEYE-X15,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2021-059AR",  # ICEYE-X11,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2022-002CQ",  # ICEYE-X14,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2022-057AD",  # ICEYE-X20,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2022-057AG",  # ICEYE-X17,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2023-001AS",  # ICEYE-X21,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2023-001BF",  # ICEYE-X27,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2023-084R",  # ICEYE-X30,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2023-084T",  # ICEYE-X23,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2023-084AF",  # ICEYE-X26,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2023-084AH",  # ICEYE-X25,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2023-174AJ",  # ICEYE-X31,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2023-174AQ",  # ICEYE-X34,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2023-174AY",  # ICEYE-X35,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2024-043C",  # ICEYE-X38,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2024-043E",  # ICEYE-X37,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2024-043F",  # ICEYE-X36,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2024-149BZ",  # ICEYE-X43,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2024-149CG",  # ICEYE-X39,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2024-149CJ",  # ICEYE-X33,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2024-149CK",  # ICEYE-X40,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2024-247H",  # ICEYE-X49,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2024-247N",  # ICEYE-X47,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2025-009CT",  # ICEYE-X42,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2025-009CV",  # ICEYE-X41,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2025-009DA",  # ICEYE-X45,
    "https://celestrak.org/NORAD/elements/gp.php?INTDES=2025-009DC",  # ICEYE-X44,
]

In [58]:
# Output file to store all TLE data
with open("combined_tle.txt", "w") as outfile:
    for url in urls:
        response = requests.get(url)
        if response.status_code == 200:
            outfile.write(response.text.strip() + "\n")  # Add double newline between TLE blocks
        else:
            print(f"Failed to fetch {url} (status: {response.status_code})")

In [59]:
satellites = load.tle_file("combined_tle.txt", reload=True, filename="satellites")
satellites # this shows the full list of satellites whose position is being calculated.

[<EarthSatellite SENTINEL-1A catalog #39634 epoch 2025-04-03 04:59:01 UTC>,
 <EarthSatellite SENTINEL-1C catalog #62261 epoch 2025-04-03 04:08:24 UTC>,
 <EarthSatellite RADARSAT-2 catalog #32382 epoch 2025-04-03 04:49:23 UTC>,
 <EarthSatellite RISAT-1 catalog #38248 epoch 2025-04-03 01:21:37 UTC>,
 <EarthSatellite COSMO-SKYMED 1 catalog #31598 epoch 2025-04-03 03:52:11 UTC>,
 <EarthSatellite COSMO-SKYMED 2 catalog #32376 epoch 2025-04-03 04:40:48 UTC>,
 <EarthSatellite COSMO-SKYMED 3 catalog #33412 epoch 2025-04-03 04:18:39 UTC>,
 <EarthSatellite ICEYE-X5 catalog #44389 epoch 2025-04-03 04:26:17 UTC>,
 <EarthSatellite ICEYE-X4 catalog #44390 epoch 2025-04-03 06:07:24 UTC>,
 <EarthSatellite ICEYE-X7 catalog #46496 epoch 2025-04-03 04:46:14 UTC>,
 <EarthSatellite ICEYE-X6 catalog #46497 epoch 2025-04-03 02:57:32 UTC>,
 <EarthSatellite XR-1 (ICEYE-X10) catalog #47507 epoch 2025-04-03 03:41:41 UTC>,
 <EarthSatellite ICEYE-X8 catalog #47510 epoch 2025-04-03 05:05:50 UTC>,
 <EarthSatellite I

For sudden-onset emergencies, the time immediately before an after an event are the most critical. We can set a time range of +2 days (48h) from a certain point in time. For the purposes of this notebook, we'll use `now` (the time when the cell is run). These techniques can also be applied in the past, if a before/after analysis is needed.

In [60]:
# Calculate the current UTC time (without microseconds), then creating a time range + 48h

ts = load.timescale()

now = datetime.now(timezone.utc).replace(microsecond=0)
time_1 = now # change this if you'd like to have a different timeframe
time_2 = now + timedelta(days=2) # change this if you'd like to have a different timeframe

The `location_iteration` function lodes the geocentric location, calculates the latitude and longitude, converts them to decimal degrees, and saves them as a point coordinate. It also checks if the satellite is sunlit at the time of calculation, which can be used as an approximation of if it is daytime below.

In [61]:
def location_iteration(timer, sat):
    geocentric = sat.at(Timescale.from_datetime(ts, timer))
    lat, lon = wgs84.latlon_of(geocentric)
    longitude = lon.degrees
    latitude = lat.degrees
    coords = Point(longitude, latitude)

    return timer, longitude, latitude, coords

Let's refine our area of interest to see when/where satellites pass overhead. This demo focuses on the area surrounding the earthquake in Myanmar in March 2024

In [62]:
from shapely.geometry import box
area = box(88, 7, 108, 28)

Then, we can iterate over the time frame. The more frequent the measurements, the slower the calculation takes.

In [63]:
timer = time_1

rows = []

while timer <= time_2:
    for sat in satellites:
        timer, longitude, latitude, coords = location_iteration(timer, sat)

        row = pd.DataFrame({'satellite': sat.name, 'timestamp': timer, 'coordinates': [coords], 'lng': longitude, 'lat': latitude}, index=[0])
        rows.append(row)    

    timer += timedelta(minutes=1) # ~ 2 minutes

satellites_df = pd.concat(rows, ignore_index=True)
satellites_df["time_string"] = satellites_df["timestamp"].dt.strftime('%Y-%m-%d %X')

We only want to look at the satellite pass overs within our AOI.

## Visualize path of satellites

We can "connect the dots" calculated in the previous step to show the path the satellite takes

In [64]:
# 1. Sort and group by satellite
satellites_df = satellites_df.sort_values(['satellite', 'timestamp'])

segments = []

In [65]:
# 2. Create line segments between consecutive positions of the same satellite
for sat, group in satellites_df.groupby('satellite'):
    group = group.reset_index(drop=True)
    for i in range(len(group) - 1):
        t0 = group.loc[i, 'timestamp']
        t1 = group.loc[i + 1, 'timestamp']
        pt0 = group.loc[i, 'coordinates']
        pt1 = group.loc[i + 1, 'coordinates']

        # Time difference check
        dt = (t1 - t0)
        if dt > pd.Timedelta("5 minutes"):  # skip gaps
            continue

        line = LineString([pt0, pt1])

        segments.append({
            'satellite': sat,
            'start_time': t0,
            'end_time': t1,
            'linestring': line,
        })

In [66]:
# 3. Convert to GeoDataFrame
path_segments = gpd.GeoDataFrame(segments, geometry='linestring', crs='EPSG:4326')
path_segments_proj = path_segments.to_crs("EPSG:3857")

In [67]:
path_segments['length_m'] = path_segments_proj.geometry.length

MAX_LENGTH = 10_000_000 # m
path_segments = path_segments[path_segments['length_m'] <= MAX_LENGTH]

In [69]:
# 4. Keep only segments that intersect AOI
path_segments['intersects_aoi'] = path_segments['linestring'].intersects(area)
path_segments_clipped = path_segments[path_segments['intersects_aoi']].copy()

In [70]:
# 5. Clip the segments to the AOI
path_segments_clipped['clipped'] = path_segments_clipped['linestring'].intersection(area)

In [71]:
path_segments_clipped = path_segments_clipped.set_geometry('clipped')

In [72]:
path_segments_clipped = path_segments_clipped.drop(columns=['linestring', 'intersects_aoi', "length_m"]).set_crs("EPSG:4326")

In [73]:
path_segments_clipped["UTC"] = path_segments_clipped["start_time"].dt.strftime('%Y-%m-%d %X')

As with the single satellite, we can create a color scale that shows the time of the satellite's pass for each line segment. In this example, white is closer to `now` and orange is closer to +48h.

In [74]:
# This creates a range from 0-1 to define our colormap.
time_norm_satellites = (path_segments_clipped.start_time - time_1) / (time_2 - time_1)

colors_satellites = apply_continuous_cmap(time_norm_satellites, Oranges_9, alpha=.5)

We can roughly estimate the width of the ground coverage of a satellite to be 10km (i.e., the width of an image is 6km). To visualize this, we can set `get_width` to be 10km.

In [75]:
layer2 = PathLayer.from_geopandas(
    path_segments_clipped,
    get_color = colors_satellites,
    get_width=15000,
    opacity=1,
    auto_highlight=True
)

# Map

In [76]:
m2 = Map(
    [layer2],
    basemap_style = basemap.CartoBasemap.DarkMatter,
    )
m2

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…